In [1]:
from csv import reader, writer
from copy import deepcopy

# following function not used anymore
def import_files(file_names): # reading the files into a 2d list without the GUI
    l = [[], [], [], [], [], []]
    count = 0
    for file_name in file_names:
        try:
            with open(file_name, 'r', encoding='latin1') as f: # why does latin work but utf8 doesn't?
                # So there's latin in the database somewhere. Fine, as long as everything is written back using the
                # 'latin1' encoding
                csv_reader = reader(f)

                for i in csv_reader:
                    l[count] += [i]
                try: l[count].remove([])
                except:pass
                try: l[count].remove('')
                except: pass

            count += 1
        except FileNotFoundError:
            print('no file named', file_name, 'found in this folder')
            input('please close the program, add the missing file and try again')
    
    return l



In [2]:
def check_file(file, key, current_list): # checks the file, and modifys the list if the file passes
    # key is the key in the GUI
    if '.csv' not in file:
        return False
    
    current_list[int(key[1])] = [] # emptying the list in case the user takes more than one try
    
    
    if key == '-2-': # the Excluded File
        with open(file, 'r', encoding='latin1') as f:
            csv_reader = reader(f)
            
            for i in csv_reader:
                current_list[int(key[1])] += [i]
            # No data validation on these at the moment
            valid = True
    elif key == '-0-': # all parts file
        with open(file, 'r', encoding='latin1') as f:
            csv_reader = reader(f)
            headers = ['id', 'default_code', 'name', 'type', 'twm_active', 'product_variant_ids/id',\
                      'categ_id/id']
            first_line = True
            for i in csv_reader:
                if first_line:
                    first_line = False
                    for j in headers:
                        if j not in i:
                            return False
                current_list[int(key[1])] += [i]
            valid = True
    elif key == '-1-': # star lord bom
        with open(file, 'r', encoding='latin1') as f:
            csv_reader = reader(f)
            count = 0
            for i in csv_reader:
                if count < 6:
                    count += 1
                    continue
                current_list[int(key[1])] += [i]
            valid = True
    elif key == '-3-': # MO's
        with open(file, 'r', encoding='latin1') as f:
            csv_reader = reader(f)
            
            first_line = True
            for i in csv_reader:
                if first_line:
                    first_line = False
                    if 'Product/Display Name' not in i or 'Product/Bill of Materials/BoM Lines' not in i:
                        return False
                current_list[int(key[1])] += [i]
        valid = True
    elif key == '-4-': # Reordering Rules
        with open(file, 'r', encoding='latin1') as f:
            csv_reader = reader(f)
            first_line = True
            for i in csv_reader:
                if first_line:
                    first_line = False
                    if 'product_id/id' not in i:
                        return False
                current_list[int(key[1])] += [i]
        valid = True
    elif key == '-5-': # Sales orders
        with open(file, 'r', encoding='latin1') as f:
            csv_reader = reader(f)
            
            first_line = True
            for i in csv_reader:
                if first_line:
                    first_line = False
                    if 'Order Lines/Product/Display Name' not in i:
                        return False
                current_list[int(key[1])] += [i]
        valid = True
    elif key == '-6-': # Purchase Orders
        with open(file, 'r', encoding='latin1') as f:
            csv_reader = reader(f)
            
            first_line = True
            for i in csv_reader:
                if first_line:
                    first_line = False
                    if 'Product/Display Name' not in i and 'Description' not in i and\
                        'Product/Product/Display Name' not in i:
                        return False
                current_list[int(key[1])] += [i]
        valid = True
    else: # this funciton must be added to accrodingly is the GUI is changed
        print('invalid key passed into check_file function')
    
    return valid

In [3]:
import PySimpleGUI as sg
def basic_gui():
    sg.theme('LightPurple')
    num_of_inputs = 7
    verified = [False for i in range(num_of_inputs)]
    existant = [False for i in range(num_of_inputs)]
    current_list = [[] for i in range(num_of_inputs)]
    width, height = 20, 1
    # the initial display for the GUI
    layout = [[sg.Text('Select the files to generate the parts list')],
              [sg.Text('All Parts Export', key='-0-', size=(width, height)), sg.Input(), sg.FileBrowse()],
              [sg.Text('Flat Star Lord BoM Export', key='-1-', size=(width, height)), sg.Input(), sg.FileBrowse()],
              [sg.Text('Excluded Parts', key='-2-', size=(width, height)), sg.Input(), sg.FileBrowse()],
              [sg.Text('Manufacturing Orders Export', key='-3-', size=(width, height)), sg.Input(), sg.FileBrowse()],
              [sg.Text('Reordering Rules Export', key='-4-', size=(width, height)), sg.Input(), sg.FileBrowse()],
              [sg.Text('Sales Order Export', key='-5-', size=(width, height)), sg.Input(), sg.FileBrowse()],
              [sg.Text('Purchase Order Export', key='-6-', size=(width, height)), sg.Input(), sg.FileBrowse()],
              [sg.Submit(), sg.Cancel(), sg.Button('Check Files', button_color=('white', 'firebrick3'))]]





    window = sg.Window('File Select', layout, finalize=True)
    window['Submit'].Update(visible=False)
    

    while True: # Event loop
        if verified[0] and verified[1]: # All parts and star lord have been verified
            window['Submit'].Update(visible=True)
        
        event, values = window.read()

        if event in (sg.WIN_CLOSED, 'Cancel', 'Submit'):
            try: # Not 100% sure why the x-out button didn't work without this, but whatever
                if event in 'Submit':
                    go = True
                else:
                    go = False
                break
            except:
                go = False
                break
        elif event in ('FileBrowse',):
            pass # doesn't happen anyway
        elif event in ('Check Files',):
            # If I remember correctly, the event is a varible that stores a string describing
            # the event, while values is a dictionary that contains the data of that event.
            # Only use the values of the dicionary if the key is a number, where the number
            # corrosponds to the object on the layout setup.
            # Then check the file that was passed in. If it's good, read it (the check_file funciton
            # does that) and turn green. Otherwise, turn red.
            for i in values:
                if str(i) in '0123456':
                    key = '-' + str(i) + '-'
                    if values[i] != '': # Need to verify file
                        current = window[key].get()
                        verified[i] = check_file(values[i], key, current_list)
                        if verified[i]:
                            window[key].update(current, text_color='green')
                        else:
                            window[key].update(current, text_color='red')

    window.close()
    
    # Only continue with the program if the user clicked submit, which sets go to True
    if go:
        return current_list
    else:
        return False

In [4]:
def generate_already_obsolete(all_parts):
    current_index = 0
    
    # collecting the headers, since I wrote this before I decided to do that for everything
    for i in all_parts[0]:
        if i == 'name':
            name_index = current_index
        elif i == 'type':
            type_index = current_index
        elif i == 'twm_active':
            active_index = current_index
        current_index += 1
    
    already_obsolete = []
    for i in all_parts:
        if '[OBSOLETE]' in i[name_index] and i[active_index] == 'False' and\
            i[type_index] == 'Storable Product':
            already_obsolete += [i]
    print(len(already_obsolete), 'parts were already marked obsolete')
    return already_obsolete

In [5]:
def get_active_parts(starLord):# extracting active parts from the Star Lord BoM
    
    count = 0
    l = []
    
    for i in starLord:
        if count < 6: # the first 6 lines don't have parts in them
            count += 1
            continue
        
        count += 1
        l += [i[0].split(']')[0][1:]] # indexing to get the part number out of the name
        
    return l

In [6]:
# returns a new part list where things that aren't parts or finished goods are excluded
def exclude_by_category(parts):
    cat_parts = 'product.product_category_all' # odoo's name for 'parts'
    cat_goods = '__export__.product_category_26_48ed6423' # odoo's name for 'finished goods'
    
    # getting the appropriate header
    cat_index = 0
    for i in parts[0]:
        if i == 'categ_id/id':
            break
        else:
            cat_index += 1
    
    l = [parts[0]] + [i for i in parts if i[cat_index] == cat_parts or i[cat_index] == cat_goods]
    return l
    

In [7]:
def remove_excluded_parts(excluded_parts, obsolete_parts, active_parts, header_index):
    l = []
    
    # getting the part number out if the user put the part name into the column
    for i in excluded_parts:
        if ']' in i[0]:
            l += [ i[0].split(']')[0][1:] ]
        else:
            l += [i[0]]
    
    while '' in l:
        l.remove('')
    
    # for every part that's excluded, iterate through every part and make sure it's there
    # if it is, add it to removing so it gets removed later
    # otherwise, put it in extra parts so the user knows there are parts in excluded that 
    # didn't get excluded
    extra_parts = []
    s = set(l) # Use it later
    removing = []
    for i in l:
        found = False
        for j in obsolete_parts:
            if i in j[header_index[1]]:
                removing += [j]
                found = True
        if not found and not i in active_parts:
            extra_parts += [i] # parts in the excluded file that weren't found in All Parts
    
    
    for i in removing:
        try:
            obsolete_parts.remove(i)
        except: # I've been told not to do this... I can't remember why I decided to do this
            pass
    return s, extra_parts

In [8]:
def update_part_names(obsolete_parts, header_index):
    for i in obsolete_parts:
        if i[header_index[0]] == 'id': # skiping the headers
            continue
    
        # name adjustment
        if '[OBSOLETE] ' not in i[header_index[2]]:
            i[header_index[2]] = '[OBSOLETE] ' + i[header_index[2]]

        # type adjustment
        i[header_index[3]] = 'Storable Product'

        # Active adjustment
        i[header_index[4]] = 'False'

In [9]:
def read_MO_list(mrp_list):
    if mrp_list == []:
        print('No MO\'s found')
        return {}
    
    # getting headers... I think I came up with better ways of doing this over time and this is one
    # of the worst
    count = 0
    for i in mrp_list[0]:
        if i == 'Product/Display Name':
            assy_index = count
        elif i == 'Product/Bill of Materials/BoM Lines':
            parts_index = count
        
        count += 1
    
    try: assy_index
    except: assy_index = None
    
    try: parts_index
    except: parts_index = None
    
    # the check_file means that these shouldn't ever happen
    if not assy_index and not parts_index:
        print('I\'m not finding either of the two necessary categories in the MO file')
        input('Please add "Product/Display Name" & "Product/Bill of Materials/BOM Lines" to the "mrp.production.csv" file')
        return {}
    elif not assy_index:
        print('I\'m not finding the "Product/Display Name" in the MO file')
        input()
    elif not parts_index:
        print('I\'m not finding the "Product/Bill of Materials/BOM Lines" in the MO file')
        input()
    
    
    mrp_active_assys, mrp_active_parts = [], []
    # making lists of active assys and parts
    first_line = True
    for i in mrp_list:
        if first_line:
            first_line = False
            continue
        if i[assy_index] != '':
            mrp_active_assys += [[i[assy_index].split(']')[0][1:], i[assy_index]]]
        if i[parts_index] != '':
            mrp_active_parts += [[i[parts_index].split(']')[0][1:], i[parts_index]]]
    
    # changing those into dictionaries, where the pn is the key, and the values are a list where
    # the first value is the number of times it's been seen and the second is the entire name
    d = {}
    for j in [mrp_active_assys, mrp_active_parts]:
        for i in j:
            if i[0] not in d: # if the part number isn't in the dictionary, add it
                d[i[0]] = [1, i[1]]
            else: # otherwise, add one to the count of the number of times that part has been seen
                d[i[0]][0] += 1
    try:
        d.pop('')
    except:
        pass
    return d

In [10]:
def read_sales_order(so_list):
    if len(so_list) == 0:
        print('No SO\'s found')
        return {}
    
    # getting the header
    name_index = so_list[0].index('Order Lines/Product/Display Name')
    
    # loading every part in the SO into a dictionary, with the same format as the dictionary
    # in the MO function
    d = {}
    first_line = True
    for i in so_list:
        if first_line:
            first_line = False
        elif i[name_index] == '0':
            continue
        else:
            if ']' in i[name_index]:
                if i[name_index].split(']')[0][1:] not in d:
                    d[i[name_index].split(']')[0][1:]] = [1, i[name_index]]
                else:
                    d[i[name_index].split(']')[0][1:]][0] += 1
            else:
                if i[name_index] not in d:
                    d[i[name_index]] = [1, i[name_index]]
                else:
                    d[i[name_index]][0] += 1
    try:
        d.pop('')
    except:
        pass
    
    return d
            

In [11]:
def read_purchase_order(po_list):
    if len(po_list) == 0:
        print('No PO\'s found')
        return {}
    
    # there's more than one possible name here. My dumbass decided to find the used one with exceptions
    try: name_index = po_list[0].index('Product/Display Name')
    except:
        try: name_index = po_list[0].index('Description')
        except: name_index = po_list[0].index('Product/Product/Display Name')
    
    # same dictionary setup as the MO and SO
    d = {}
    first_line = True
    for i in po_list:
        if first_line:
            first_line = False
        else:
            if i[name_index].split(']')[0][1:] not in d:
                d[i[name_index].split(']')[0][1:]] = [1, i[name_index]]
            else:
                d[i[name_index].split(']')[0][1:]][0] += 1
    try:
        d.pop('')
    except:
        pass
    return d

In [12]:
def write_questionable_file(l):
    # l is a four element list containing the consumable parts, SO's, PO's, and MO's. Each are their
    # own list, which are four wide by however many parts there are deep.
    
    # changing the order of l so it's sorted by length, from longest to shortest
    order_key = [[len(i[0]), i] for i in l]
    def sort_func(l):
        return l[0]
    order_key.sort(key=sort_func, reverse=True)
    ordered_l = [i[1][0] for i in order_key]
    header_order = [i[1][1] for i in order_key]
    
    # writing the headers for the file
    header_text = []
    for i in header_order:
        if i in ['consumable']:
            header_text += ['Consumable Part Numbers', 'Consumable Part Names', 'Part type', '']
        elif i in ['mrp']:
            header_text += ['MO Part Numbers', 'MO Part Names', 'Number found in MO\'s', '']
        elif i in ['so']:
            header_text += ['SO Part Numbers', 'SO Part Names', 'Number found in SO\'s', '']
        elif i in ['po']:
            header_text += ['PO Part Numbers', 'PO Part Names', 'Number found in PO\'s', '']
        else:
            print('unrecognized header code: ', i)
    # writing the file
    while True:
        try:
            with open('questionable.csv', 'w', encoding='latin1') as f:
                csv_writer = writer(f)
                csv_writer.writerow(header_text)

                for i in range(1, len(ordered_l[0])):
                    line = ordered_l[0][i]
                    for j in range(1, 4): # only adding the next column to the line if it exists
                        try:
                            line += ordered_l[j][i]
                        except IndexError:
                            break # since it's sorted, if one doesn't exist, the rest won't either.
                    csv_writer.writerow(line)
            break
        except PermissionError:
            print('I couldn\'t access the questionable file. Make sure it isn\'t open in Excel or',\
                 'any other file viewer/editor. Press Enter when ready.')
            input()

In [13]:
def write_big_ssheet(obsolete_parts):
    while True:
        try:
            with open('obsolete_update.csv', 'w', encoding='latin1') as f:
                csv_writer = writer(f)
                for i in obsolete_parts:
                    csv_writer.writerow(i)
            break
        except PermissionError:
            print('I couldn\'t access the obsolete_update file. Make sure it isn\'t open in Excel or',\
                 'any other file viewer/editor. Press Enter when ready')
            input()

In [14]:
def deal_with_reordering_rules(obsolete_parts, ror_list, header_index):
    if len(ror_list) == 0:
        print('Not doing reorder rules')
        return
    
    # reading the reorder rules export to find the ones that exist
    d = {}
    first_line = True
    for i in ror_list:
        if first_line:
            first_line = False
            index = 0
            for j in i: # I guess I forgot that .index() is a thing?
                if j == 'product_id/id':
                    product_id_found = True
                    ext_id_index = index
                elif j == 'id':
                    id_index = index
                index += 1
            if not product_id_found:
                print('make sure to include the Product/Name category in the Reordering rules export')
                return
        else:
            d[i[ext_id_index]] = i[id_index]
    
    # DNE -> does not exist. ror -> reordering rule
    ror_exists, ror_DNE = [], []
    for i in obsolete_parts[1:]:
        if i[header_index[5]] in d:
            ror_exists += [i]
        else:
            ror_DNE += [i]
    
    # the parts that already have a reordering rule
    while True:
        try:
            with open('Updated Reordering Rules.csv', 'w', encoding = 'latin1') as f:
                csv_writer = writer(f)
                csv_writer.writerow(['id', 'product_max_qty', 'product_min_qty', 'qty_multiple'])

                for i in ror_exists:
                    csv_writer.writerow([d[i[header_index[5]]], 0, 0, 1])
            break
        except PermissionError:
            print('I couldn\'t open the Updated Reordering Rules file. Make sure it isn\'t open in',\
                 'Excel, or any other file viewer/editor. Press Enter when ready.')
            input()
    
    # the parts that need a reordering rule created
    while True:
        try:
            with open('New Reordering Rules.csv', 'w', encoding = 'latin1') as f:
                csv_writer = writer(f)
                csv_writer.writerow(['product_id/id', 'name', 'product_max_qty', 'product_min_qty', 'qty_multiple'])
                for i in ror_DNE:
                    if not ( i[header_index[1]][:2] == '20' and \
                           (i[header_index[1]][4] == 'A' or i[header_index[1]][4] == 'E')):# not writing assys
                        csv_writer.writerow([i[header_index[5]], i[header_index[2]], 0, 0, 1])
            break
        except PermissionError:
            print('I couldn\'t access the New Rordering Rules file. Make sure it isn\'t open in Excel',\
                 'or any other file viewer/editor. Press Enter when ready.')
            input()

In [ ]:
def main():
    input_file_names = ['product.template.csv', 'bom_structure_and_cost.csv', \
                        'mrp.production.csv', 'Excluded Parts.csv', 'sale.order.csv',\
                        'purchase.order.line.csv'] # doesn't use these anymore
    #data = import_files(input_file_names)
    data = basic_gui()
    if not data:
        print('goodbye (canceled/closed window)')
        return
    # indexing is the same order as the above list, and as follows:
    # 0 is all parts
    # 1 is star lord BoM (all active parts)
    # 2 is excluded parys
    # 3 is MO's
    # 4 is reordering rules
    # 5 is SO's
    # 6 is PO's
    # all except for 1 and 2 are optional...
    
    
    all_parts = data[0]
    while [] in all_parts:
        all_parts.remove([])
    
    # finding the headers, and storing them in a list so they don't have to be re-found
    defined_headers = ['id', 'default_code', 'name', 'type', 'twm_active', 'product_variant_ids/id',\
                       'categ_id/id']
    current_index = 0
    header_index = [9 for i in range(7)]
    for i in all_parts[0]:
        if i in defined_headers:
            header_index[defined_headers.index(i)] = current_index
        current_index += 1
    
    already_obsolete = generate_already_obsolete(all_parts)
    active_parts = get_active_parts(data[1])
    active_parts.remove('')
    active_parts = set(active_parts)
    
    # creating the obsolete parts list, which has the same format as the All_parts upload.
    already_obsolete_pns = set([i[header_index[1]] for i in already_obsolete])
    obsolete_parts = []
    not_storable = []
    tm_only = False # change to True to only include Twin Monkeys Parts
    for i in all_parts:
        try:
            if tm_only:
                if i[header_index[1]] not in active_parts and \
                    i[header_index[1]][0:2] == '20' and (\
                    i[header_index[1]][4] == 'D' or i[header_index[1]][4] == 'F' or\
                     i[header_index[1]][4] == 'E' or i[header_index[1]][4] == 'A'):

                    obsolete_parts += [i]

                    if i[header_index[3]] != 'Storable Product':
                        not_storable += [i]
            else:
                if i[header_index[1]] not in active_parts and \
                   i[header_index[1]] not in already_obsolete_pns:
                    obsolete_parts += [i]

                if i[header_index[3]] != 'Storable Product':
                    not_storable += [i]     
        except:
            pass
    not_storable = deepcopy(not_storable)
    
    excluded_pns, extra_parts = remove_excluded_parts(data[2], obsolete_parts, \
                                                      active_parts, header_index)
    
    # if it's excluded, it doesn't need to show up on the questionable.csv
    removing = []
    for i in not_storable:
        if i[header_index[1]] in excluded_pns:
            removing += [i]
    for i in removing:
        not_storable.remove(i)
    
    obsolete_parts = exclude_by_category(obsolete_parts)
    not_storable = exclude_by_category(not_storable)
    update_part_names(obsolete_parts, header_index)
    
    mrp_active = read_MO_list(data[3])
    so_active = read_sales_order(data[5])
    po_active = read_purchase_order(data[6])
    obsolete_active_mrp = []
    obsolete_active_so = []
    obsolete_active_po = []
    # turning the dictionaries from the orders into part lists
    for i in obsolete_parts:
        if i[header_index[1]] in mrp_active:
            obsolete_active_mrp += [mrp_active[i[header_index[1]]]]
        if i[header_index[1]] in so_active:
            obsolete_active_so += [so_active[i[header_index[1]]]]
        if i[header_index[1]] in po_active:
            obsolete_active_po += [po_active[i[header_index[1]]]]
    
    # formatting the lists to the way they'll be written to the file
    for i in obsolete_active_mrp:
        index = obsolete_active_mrp.index(i)
        obsolete_active_mrp[index] = [i[1].split(']')[0][1:], \
                                      i[1].split(']')[1][1:], i[0], '']
    for i in obsolete_active_so:
        index = obsolete_active_so.index(i)
        obsolete_active_so[index] = [i[1].split(']')[0][1:], \
                                      i[1].split(']')[1][1:], i[0], '']
    for i in obsolete_active_po:
        index = obsolete_active_po.index(i)
        obsolete_active_po[index] = [i[1].split(']')[0][1:], \
                                      i[1].split(']')[1][1:], i[0], '']
    
    consumable_ish = [] # not_storable was already used and needed to be reformatted
    for i in not_storable:
        consumable_ish += [[i[header_index[1]], i[header_index[2]], i[header_index[3]], '']]
    write_questionable_file([[obsolete_active_mrp, 'mrp'], [consumable_ish, 'consumable'],\
                             [obsolete_active_so, 'so'], [obsolete_active_po, 'po']])
    write_big_ssheet(obsolete_parts)
    deal_with_reordering_rules(obsolete_parts, data[4], header_index)
    
    # displaying results
    print(len(obsolete_parts)-1, ' parts of ', len(all_parts)-1, \
          ' parts(', 100*(len(obsolete_parts)-1)/len(all_parts)-1,\
          '%) were marked obsolete', sep='' )# the -1 is to exclude the headers
    
    print('Parts that were marked to be excluded but weren\'t found: ')
    for i in extra_parts:
        print(i)
    if len(extra_parts) == 0:
        print('None')
    print('\n')
    
    input('all done') # not closing the program so the user can read anything that's printed

# formatted weirdly for error catching. And it's easy to put it in a more debugging-friendly setup
not_compiled = '''
main()
compiled = '''
try:
    main()
except BaseException as err: # ha. I handle errors more better than solidworks
    print(f"Unexpected {err=}, {type(err)=}")
    input('goodbye (errored out)')
    raise # if the user presses enter the program just exits
    ''''''

3466 parts were already marked obsolete
No MO's found
No SO's found
No PO's found
Not doing reorder rules
1 parts of 8079 parts(-0.9876237623762376%) were marked obsolete
Parts that were marked to be excluded but weren't found: 
SK-111


